In [1]:
using Pkg
Pkg.activate("libs/")
using Lux, Random, NNlib, Zygote, LuxCUDA, CUDA, FluxMPI, JLD2, DICOM
using Images
using ImageView
using MLUtils
using Optimisers, Statistics

train_output_dir = "/media/molloi-lab/System Files/BAC_dataset/dataset_for_ML_full";

patch_size = 256
patch_size_half = round(Int, patch_size/2)

total_num_patch = 49772+49910+50154+49980+49963+50039+49561+49952

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


┌ Warning: MPI Implementation is not CUDA Aware.
└ @ FluxMPI /home/molloi-lab/.julia/packages/FluxMPI/OM5f6/src/FluxMPI.jl:28


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/molloi-lab/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/molloi-lab/snap/code/common/.cache/gio-modules/libgiolibproxy.so


399331

# 1. Load data into container

In [2]:
isdir("/media/molloi-lab/System Files/BAC_dataset/dataset_for_ML_full")

true

In [3]:
dirs = []
for f1 in readdir(train_output_dir)
    p1 = joinpath(train_output_dir, f1)
    for f2 in readdir(p1)
        p2 = joinpath(p1, f2)
        tmp_dir = []
        for f3 in readdir(p2)
            p3 = joinpath(p2, f3)
            push!(tmp_dir, p3)
        end
        push!(dirs, tmp_dir)
    end
end

In [4]:
image_paths = Array{String, 1}(undef, total_num_patch)
label_paths = Array{String, 1}(undef, total_num_patch)
ct = 0
for dir_ in dirs
    img_dir, lbl_dir = dir_
    for f in readdir(img_dir)
        image_paths[ct+=1] = joinpath(img_dir, f)
        label_paths[ct] = joinpath(lbl_dir, f)
    end
end

In [5]:
# shuffle
random_indices = randperm(total_num_patch)
image_paths = image_paths[random_indices]
label_paths = label_paths[random_indices];

In [6]:
each_gpu_num_patches = round(Int, total_num_patch/4)

99833

In [7]:
b_s = 6
GC.gc(true)
# split into 4 gpus
for i = 1 : 4
    @info i
    start_idx = 1 + each_gpu_num_patches*(i-1)
    end_idx = i==4 ? total_num_patch : each_gpu_num_patches*i
    curr_num = end_idx - start_idx + 1
    
    # container
    train_container_images = Array{Float32, 4}(undef, patch_size, patch_size, 1, curr_num)
    train_container_masks = Array{Float32, 4}(undef, patch_size, patch_size, 1, curr_num)

    Threads.@threads for i = start_idx : end_idx
        idx = i-start_idx+1
        p1, p2 = image_paths[i], label_paths[i]
        @load p1 image_patch
        train_container_images[:, :, 1, idx] = image_patch
        @load p2 label_patch
        train_container_masks[:, :, 1, idx] = label_patch
    end

    # data loader
    train_loader = MLUtils.DataLoader((data=train_container_images, label=train_container_masks), batchsize=b_s)
    @save "train_loader_$i.jld2" train_loader
    train_loader = nothing
    train_container_images = nothing
    train_container_masks = nothing
    GC.gc(true)
end

┌ Info: 1
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/1.5_create_data_loader.ipynb:5


┌ Info: 2
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/1.5_create_data_loader.ipynb:5


┌ Info: 3
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/1.5_create_data_loader.ipynb:5


┌ Info: 4
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/1.5_create_data_loader.ipynb:5
